# Mult agent systems
- autogen_agentchat.teams : This module provides implementation of various pre defined multi agent teams. Each team inherits from. the BaseGroupChat class
- Refer to documentation once 
- Why multi agent over single agent - a) Specialization b) Agents can work on async manner c) Resiliency - if an agent is lazy or flop, others can pick up  

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4', api_key=api_key)

In [3]:
from autogen_agentchat.agents import AssistantAgent

story_agent = AssistantAgent(
    name = 'story_agent',
    model_client = model_client,
    system_message = 'you are a content writer. Generate stories on any given topic'
)

from autogen_agentchat.messages import TextMessage

async def test_simple_agent():
    task = TextMessage(
        content = "Write a short story about two little girls and a pink cat in 300 words",
        source = 'user'
    )
    result = await story_agent.run(task= task)
    print(result.messages[-1].content)
await test_simple_agent()

Once upon a time, in the heart of a quiet town nestled in the midst of snow-capped mountains and lush green fields, lived two adorable little girls named Lucy and Ella. They were not just sisters, but best friends who shared an irrepressible knack for finding everyday adventures.

One delightful summer afternoon, while wandering through Mrs. Hoot's Sunflower field, they found something that would take their expeditions to a whole new level. In the midst of the towering sunflowers, a petite, fluffy creature peeked back at them with curious, glittering emerald eyes. To their surprise, it was a cat, an extraordinary one at that - it had the most beautiful, shiny pink fur they had ever seen.

Confused yet awestruck, the girls named their newfound feline friend 'Rosie' after its unique rose-colored coat. The girls could hardly contain their excitement at the idea of sharing their everyday escapades with Rosie. It wasn't long before Rosie the pink cat became an integral figure in Lucy and El

# Multi Agent team approach 


In [4]:
from autogen_agentchat.messages import TextMessage

plot_agent = AssistantAgent(
    name = 'plot_agent',
    model_client = model_client,
    system_message = "You create engaging plots for stories for a story topic"
)

character_agent = AssistantAgent(
    name = 'character_agent',
    model_client = model_client,
    system_message = "You develop characters and their behaviors for a given story topic"
)

ending_agent = AssistantAgent(
    name = 'ending_agent',
    model_client = model_client,
    system_message = "You write ending of a story. Always conclude stories with twists"
)

from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [plot_agent,character_agent, ending_agent],
    max_turns = 6
)

async def test_team():
    task = TextMessage(
        content = "Write a short story about two little girls and a pink cat. Keep it to 300 words",
        source = 'user'
    )
    result = await team.run(task = task)
    for each_agent_message in result.messages:
        print(f'{each_agent_message.source}: {each_agent_message.content}')
    
await test_team()




user: Write a short story about two little girls and a pink cat. Keep it to 300 words
plot_agent: Once, in a small countryside town, lived two kindred spirits, Lila and Rosie. Merely aged ten, they were playful, adventurous, and kind. Their bond was a unique flair in town; it was facilitated by their shared love for cats. This love took a mystic turn when they encountered a Pink Cat in the forest.

One sunny afternoon, Lila and Rosie were exploring the dense forest. Suddenly, a shimmer of pink caught their eye; a fluffy cat, the most unusual shade of pink, was resting by a crystal-clear stream. It was no ordinary creature; the cat exuded an enchanting aura. They named it 'Blossom'.

Blossom became an integral part of their adventures. It was as if they were always destined to meet Blossom, forming a beautiful yet peculiar trio. The girls, guided by Blossom, started solving minor problems of the town, from finding lost objects to helping the old Mrs. Crumbley find her way home when she 

In [ ]:
import os
import sys
import asyncio
from typing import List
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Load environment variables
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Initialize the model client
llm_client = OpenAIChatCompletionClient(model='gpt-4', api_key=api_key)

# Create agents
user_proxy = UserProxyAgent(
    name="user"
)

search_agent = AssistantAgent(
    name="web_searcher",
    system_message=(
        "You are a web searcher. When asked, do a concise web-style search summary: list 3 latest findings "
        "or facts with sources (title and short snippet). If no web access, respond with best known facts."
    ),
    model_client=llm_client,
    description="Searches for information and provides summaries with sources"
)

analysis_agent = AssistantAgent(
    name="analyst",
    system_message=(
        "You are an analyst. Take the web searcher output and produce a 3 bullet next step action summary "
        "and suggest one immediate follow-up question for the user to get deep into the topic."
    ),
    model_client=llm_client,
    description="Analyzes information and provides actionable insights"
)

# Create termination conditions
term_condition = MaxMessageTermination(max_messages=12)


# Create the SelectorGroupChat
groupchat = SelectorGroupChat(
    participants=[user_proxy, search_agent, analysis_agent],
    model_client=llm_client,  # Required: model client for the selector
    termination_condition=term_condition,
    max_turns=20,
    allow_repeated_speaker=False,
    selector_prompt="Select the most appropriate agent to handle the next part of the task based on the conversation context and agent descriptions."
)

# Create UI console
import sys
ui = Console(stream=sys.stdout)

async def main():
    """Main async function to run the group chat."""
    try:
        # Run the group chat with the initial task
        task = "Please summarize the developments in India US relationship in 2025 and suggest implications and next steps"
        
        # Start the conversation
        result = await groupchat.run(task=task, cancellation_token=None)
        
        print("\n" + "="*50)
        print("FINAL CONVERSATION SUMMARY")
        print("="*50)
        
        # Print the conversation history
        for i, message in enumerate(result.messages, 1):
            print(f"\n{i}. {message.source}: {message.content}")
        
        print(f"\nConversation ended. Reason: {result.stop_reason}")
        
    except Exception as e:
        print(f"Error running group chat: {e}")

if __name__ == "__main__":
    # Run the async main function
    await main()

/var/folders/9d/7gw2rnq56ls5m4r1vbglz5r00000gn/T/ipykernel_51737/3448188332.py:60: RuntimeWarning: coroutine 'Console' was never awaited
  ui = Console(stream=sys.stdout)
Model failed to select a speaker after 3, using the previous speaker.


# Observing teams in Autogen 

In [2]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

plot_agent = AssistantAgent(
    name = 'plot_agent',
    model_client = model_client,
    system_message = "You create engaging plots for stories for a story topic"
)

character_agent = AssistantAgent(
    name = 'character_agent',
    model_client = model_client,
    system_message = "You develop characters and their behaviors for a given story topic"
)

ending_agent = AssistantAgent(
    name = 'ending_agent',
    model_client = model_client,
    system_message = "You write ending of a story. Always conclude stories with twists"
)

from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [plot_agent,character_agent, ending_agent],
    max_turns = 3
)

In [11]:
from autogen_agentchat.base import TaskResult

async for message in team.run_stream(task = 'Write a story on a king , a queen and her lover'):
    if isinstance(message,TaskResult):
        print(f"Stop Reason: {message.stop_reason}")
    else:
        print((message))

id='45ef76d5-9e0e-4e84-9ef4-264f1b56d2f2' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 8, 23, 27, 21, 471789, tzinfo=datetime.timezone.utc) content='Write a story on a king , a queen and her lover' type='TextMessage'
id='548e21ed-7637-43fd-8a59-f8ed65b69b48' source='plot_agent' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=884) metadata={} created_at=datetime.datetime(2025, 8, 8, 23, 27, 43, 209894, tzinfo=datetime.timezone.utc) content='Title: The Royal Triangle\n\nIn the heart of the lush kingdom of Verdonia, where the rivers sang harmonious melodies and the forests whispered secrets, reigned King Alaric. Known for his wisdom and valor, Alaric was a beloved ruler who stood as a pillar of strength for his subjects. Beside him was Queen Elara, the epitome of grace and kindness, whose beauty was rivaled only by her compassionate heart.\n\nHowever, hidden behind the ceremonial smiles and regal façade lay a love story cloaked in shadows

In [3]:
from autogen_agentchat.ui import Console

await Console(team.run_stream(task = 'Write a story on a king , a queen and her lover in 200 words'))

---------- TextMessage (user) ----------
Write a story on a king , a queen and her lover in 200 words
---------- TextMessage (plot_agent) ----------
In the ancient kingdom of Elandor, King Alaric was loved for his wisdom and fairness, while Queen Seraphina was admired for her beauty and gentle heart. Yet, behind the serene facade of the castle, a secret affair unraveled, threatening the sanctity of the crown.

Seraphina, trapped in a loveless marriage, found solace in the arms of Thorne, a dashing knight of humble origins. They had grown up together in the lush meadows of her father's lands, their bond forged in innocence and true affection. As adults, their love blossomed in secret meetings beneath the ancient willow by the riverbank, hidden from the prying eyes of the court.

Unbeknownst to the queen, a jealous count had uncovered their rendezvous. Seeking favor and power, he whispered his discovery into the king's ear. Alaric, torn between his duty and love for Seraphina, hatched a 

TaskResult(messages=[TextMessage(id='a2a4709c-30eb-433c-bfeb-27b6fb1cfe68', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 21, 4, 18, 25, 773688, tzinfo=datetime.timezone.utc), content='Write a story on a king , a queen and her lover in 200 words', type='TextMessage'), TextMessage(id='48134419-376d-46fb-971f-77be64211a35', source='plot_agent', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=287), metadata={}, created_at=datetime.datetime(2025, 9, 21, 4, 18, 30, 841025, tzinfo=datetime.timezone.utc), content="In the ancient kingdom of Elandor, King Alaric was loved for his wisdom and fairness, while Queen Seraphina was admired for her beauty and gentle heart. Yet, behind the serene facade of the castle, a secret affair unraveled, threatening the sanctity of the crown.\n\nSeraphina, trapped in a loveless marriage, found solace in the arms of Thorne, a dashing knight of humble origins. They had grown up together in the lush meadows of he

# Team operations   
- Resume

In [1]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

from autogen_agentchat.agents import AssistantAgent
add_1_agent_first = AssistantAgent(
    name = 'add_1_agent_first',
    model_client=model_client,
    system_message="Add 1 to the number, first number is 0. Give result as output"
)

add_1_agent_second = AssistantAgent(
    name = 'add_1_agent_second',
    model_client=model_client,
    system_message="Add 1 to the number. Give result as output."
)
 
add_1_agent_third = AssistantAgent(
    name = 'add_1_agent_third',
    model_client=model_client,
    system_message="Add 1 to the number. Give result as output."
)

from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third],
    max_turns=5
)

from autogen_agentchat.ui import Console

await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
2
---------- TextMessage (add_1_agent_third) ----------
3
---------- TextMessage (add_1_agent_first) ----------
4
---------- TextMessage (add_1_agent_second) ----------
5


TaskResult(messages=[TextMessage(id='ae8288d2-6bb5-40e3-90ed-f30815dadf4d', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 33, 30, 141772, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='4f193bea-ba77-4078-bd36-894d16ce61ef', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 33, 30, 679969, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(id='a191ec29-ea09-4896-a9c1-40c763c85a2e', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=39, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 33, 31, 316653, tzinfo=datetime.timezone.utc), content='3', type='TextMessage'), TextMessage(id='7def664d-0b02-4938-95ee-2b42e5ccde5d', source='add_1_agent_first', models_usage=RequestUsage(prompt_t

In [2]:
await Console(team.run_stream())

---------- TextMessage (add_1_agent_third) ----------
6
---------- TextMessage (add_1_agent_first) ----------
7
---------- TextMessage (add_1_agent_second) ----------
8
---------- TextMessage (add_1_agent_third) ----------
9
---------- TextMessage (add_1_agent_first) ----------
10


TaskResult(messages=[TextMessage(id='ef85d820-591f-4d17-b86e-ffa8a08b45de', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=64, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 33, 49, 25538, tzinfo=datetime.timezone.utc), content='6', type='TextMessage'), TextMessage(id='8b1d7501-1cd1-419b-821f-3d5e3f9d3f3b', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=76, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 33, 49, 654287, tzinfo=datetime.timezone.utc), content='7', type='TextMessage'), TextMessage(id='7f10e4c4-984c-44c3-b3fa-ff05cc2a6e6a', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=81, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 33, 50, 253617, tzinfo=datetime.timezone.utc), content='8', type='TextMessage'), TextMessage(id='43e39ca0-df21-4412-a9b1-13955dde640d', source='add_1_agent_third', models_usage=RequestUsage(prompt_to

In [3]:
await Console(team.run_stream())

---------- TextMessage (add_1_agent_second) ----------
11
---------- TextMessage (add_1_agent_third) ----------
12
---------- TextMessage (add_1_agent_first) ----------
13
---------- TextMessage (add_1_agent_second) ----------
14
---------- TextMessage (add_1_agent_third) ----------
15


TaskResult(messages=[TextMessage(id='a30ed479-79e9-498a-9a55-2db60407ccce', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=107, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 33, 55, 936607, tzinfo=datetime.timezone.utc), content='11', type='TextMessage'), TextMessage(id='1c92527c-0255-4330-9a0e-62e573f1559d', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=114, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 33, 56, 509788, tzinfo=datetime.timezone.utc), content='12', type='TextMessage'), TextMessage(id='cb393f88-4731-436e-8fd1-c6db25fa1dc7', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 33, 57, 128844, tzinfo=datetime.timezone.utc), content='13', type='TextMessage'), TextMessage(id='70e57281-0994-41b9-bf93-7e1284419607', source='add_1_agent_second', models_usage=RequestUsage(p

- Reset operation

In [4]:
await team.reset() # on_reset() on all agents

In [5]:
await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
2
---------- TextMessage (add_1_agent_third) ----------
3
---------- TextMessage (add_1_agent_first) ----------
4
---------- TextMessage (add_1_agent_second) ----------
5


TaskResult(messages=[TextMessage(id='5eb379f5-c555-4cab-9857-97d625fb36ac', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 34, 27, 841270, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='1c3d3881-0eef-4417-928f-b1de514eced1', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 34, 28, 613324, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(id='94f42489-f3f9-4cfb-b178-aa9e1c1829c9', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=39, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 9, 4, 34, 29, 207656, tzinfo=datetime.timezone.utc), content='3', type='TextMessage'), TextMessage(id='b94568f3-2257-4a5a-ad28-e4097decc34b', source='add_1_agent_first', models_usage=RequestUsage(prompt_t

- Termination conditions

In [ ]:
from autogen_agentchat.agents import AssistantAgent
add_1_agent_first = AssistantAgent(
    name = 'add_1_agent_first',
    model_client=model_client,
    system_message="Add 1 to the number, first number is 0. Give result as output"
)

add_1_agent_second = AssistantAgent(
    name = 'add_1_agent_second',
    model_client=model_client,
    system_message="Add 1 to the number. Give result as output."
)
 
add_1_agent_third = AssistantAgent(
    name = 'add_1_agent_third',
    model_client=model_client,
    system_message="Add 1 to the number. Give result as output."
)

from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third]
)
from autogen_agentchat.ui import Console

await Console(team.run_stream())

In [ ]:
from autogen_agentchat.conditions import MaxMessageTermination

max_termination = MaxMessageTermination(5)
team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third],
    termination_condition =max_termination
)

In [19]:
# Another option
agent1 = AssistantAgent(
    name = 'story_writer',
    model_client=model_client,
    system_message="Give the story about a brave knight, keep it short no more than 40 words. if critic say 'THE END' anywhere. Only output 'THE END'"
)

agent2 = AssistantAgent(
    name = 'story_critic',
    model_client=model_client,
    system_message="Continue the story and critic it with feedback. Keep it short and no more than 40 words. If it feels complete, just say 'THE END'. Only output 'THE END'"
)
from autogen_agentchat.conditions import TextMentionTermination


text_mention_termination = TextMentionTermination('THE END')
teamWithTextTermination = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=text_mention_termination
)

from autogen_agentchat.ui import Console

await Console(teamWithTextTermination.run_stream(task = 'Write a story about one girl and a panda in 50 words.'))

---------- TextMessage (user) ----------
Write a story about one girl and a panda in 50 words.
---------- TextMessage (story_writer) ----------
A brave knight ventured through a dark forest to save his kingdom's treasure. He battled fierce beasts and tricked cunning sorcerers. Heart pure and sword steady, he reached the hidden cave, rescuing the realm's stolen fortune. His courage became legend. The kingdom celebrated his heroics.
---------- TextMessage (story_critic) ----------
THE END


TaskResult(messages=[TextMessage(id='3ba7153b-e4a9-49dd-a00f-2cdbc77fe16c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 9, 0, 14, 45, 694013, tzinfo=datetime.timezone.utc), content='Write a story about one girl and a panda in 50 words.', type='TextMessage'), TextMessage(id='5a783315-2400-43b6-b38c-76cd961cf39a', source='story_writer', models_usage=RequestUsage(prompt_tokens=59, completion_tokens=59), metadata={}, created_at=datetime.datetime(2025, 8, 9, 0, 14, 48, 158432, tzinfo=datetime.timezone.utc), content="A brave knight ventured through a dark forest to save his kingdom's treasure. He battled fierce beasts and tricked cunning sorcerers. Heart pure and sword steady, he reached the hidden cave, rescuing the realm's stolen fortune. His courage became legend. The kingdom celebrated his heroics.", type='TextMessage'), TextMessage(id='df265111-14a4-496d-9d89-e175e6a3573c', source='story_critic', models_usage=RequestUsage(prompt_tokens=129, compl